In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,classification_report

# Modélisation

In [ ]:
print(os.path.abspath(''))
file_path = input('Chemin absolu du fichier à charger : \n')
# data_avec_etiquettes.txt

In [ ]:
# Variables pertinentes
cols = ['V14', 'V84', 'V96', 'V159', 'V160', 'V161','V162','V163', 
        'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 
        'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 
        'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 
        'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 
        'V194', 'V195', 'V197', 'V198', 'V199','V200']

# variables explicatives
X = pd.read_csv(file_path,delimiter='\t',usecols=cols
[:-1])

# variable cible  
y = pd.read_csv(file_path,delimiter='\t',usecols=[cols
[-1]])   

y=y.to_numpy().reshape(-1)

# recodage des variables qualitatives
X=pd.get_dummies(X, columns=['V160','V162'],drop_first=True)
X[["V161"]]=X[["V161"]].apply(LabelEncoder().fit_transform) 

# Division apprentissage/test
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,
test_size=0.3,stratify=y,random_state=42)

# Apprentissage sur le train set
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train) # met un peu de temps
pred=dtree.predict(X_test)

# Evaluation

In [ ]:
df=pd.DataFrame({'y_test':pd.Series(y_test).value_counts(),'pred':pd.Series(pred).value_counts()})
df=df.fillna(0)
df=df.astype(int)
df

In [ ]:
print(df[df['y_test']==df['pred']].shape[0])

In [ ]:
df=pd.DataFrame({'y_test':y_test,'prediction':pred})
df[df['y_test']!=df['prediction']].shape[0]

In [ ]:
# Taux du succès
print("Taux de succès : ",accuracy_score(y_test,pred))

#Calcul du taux d'erreur
print("Taux d'erreur : "+str(1.0-accuracy_score(y_test,pred)))

In [ ]:
labels=list(set(y_test))
labels.sort()
precision = pd.DataFrame({"precision_score":precision_score(y_test,pred,average=None,labels=labels)},index=labels)
precision

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
#importance des variables
imp = {"VarName":X_train.columns,"Importance":dtree.feature_importances_}
print(pd.DataFrame(imp).sort_values(by="Importance",ascending=False))

In [ ]:
from sklearn.tree.export import export_text
r=export_text(dtree,feature_names = list(X_train.columns))
print(r)